In [ ]:
%matplotlib notebook
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}

import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from source.spectral import Spectral
np.random.seed(0)

In [ ]:
plt.rcParams['figure.figsize'] = [4,4]

# Initializing spectral model

In [ ]:
spec = Spectral(1, 1, 64, 64, 0.3, 'random')
spec.step = spec.step_method5

In [ ]:
fig, axs = plt.subplots()
axs.set_title("Visualization")
axs.set_xlabel("X")
axs.set_ylabel("Y")

img=False
# levels=[-1, 0, 1]
levels=5

if img : image = axs.imshow(spec.u, cmap='gray', origin='lower', animated=True)
if not img : axs.contourf(spec.y, spec.x, spec.u, levels=levels, cmap='gray')

dt, Nt = 1e-7, 1
def animate(i):
    spec.advance(dt, Nt)

    if img: image.set_data(spec.u)
    if not img: axs.clear()
    if not img: axs.contourf(spec.y, spec.x, spec.u, levels=levels, cmap='gray')
    return

In [ ]:
anim = animation.FuncAnimation(fig=fig, func=animate)
fig.show()

In [ ]:
spec = Spectral(1, 1, 512, 512, 0.03, 'random')
spec.step = spec.step_method5

In [ ]:
"""Press Ctrl+Return on this cell to monitor the concentrations"""
temp = f"Total: {spec.u.size}", f"Below viable: {spec.u[spec.u>1].size}", f"Above viable: {spec.u[spec.u<-1].size}"
print(temp)

Plotting $|k|^4$.

Its values depend only on the positions of the spatial grid.

In [ ]:
plot = 0
# 1: filled contour, 2: just level curves, else: greyscale image
plt.xlabel("x interval")

if plot==1:
    plt.contourf(spec.y, spec.x, spec.k2**2, cmap=plt.cm.gray)
elif plot==2:
    plt.contour(spec.y, spec.x, spec.k2**2, cmap=plt.cm.gray)
else:
    plt.imshow(spec.k2**2, extent=(0,1,0,1), cmap=plt.cm.gray, origin='lower')
plt.show()